# Flask GUI for Graph Colouring Problem Solver


In [1]:
%pip install flask networkx matplotlib



^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from flask import Flask, render_template, request, jsonify
import networkx as nx
import matplotlib.pyplot as plt
import io
import base64
import time
from Backtracking import find_optimal_backtracking
from BacktrackHeu import graph_colouring_optimized
from CulturalAlgorithm import CulturalAlgorithm

app = Flask(__name__)

COLOR_MAP = [
    "#DA42A2", "#60CF60", "#4A6EB8", "#FFFF00",
    "#800080", "#FFA500", "#00FFFF", "#FF00FF",
    "#00FF00", "#FF0000", "#A52A2A", "#FFC0CB"
]

def calculate_conflicts(graph, individual):
    conflicts = 0
    n = len(graph)
    for i in range(n):
        if individual[i] == -1:
            continue
        for j in range(i + 1, n):
            if graph[i][j] == 1 and individual[i] == individual[j]:
                conflicts += 1
    return conflicts

def calculate_chromatic_number(solution):
    if solution is None:
        return 0
    return len(set(c for c in solution if c != -1))

def plot_graph_step(n, edges, coloring, node_domains=None, step_number=None):
    canvas = nx.Graph()
    for i in range(n):
        canvas.add_node(i)
    for u, v in edges:
        canvas.add_edge(u, v)

    pos = nx.spring_layout(canvas, seed=42)
    plt.figure(figsize=(6, 6), facecolor='white')

    node_colors = []
    for c in coloring:
        if c != -1:
            node_colors.append(COLOR_MAP[c % len(COLOR_MAP)])
        else:
            node_colors.append('#e0e0e0')

    nx.draw(canvas, pos, with_labels=False,
            node_color=node_colors, node_size=700, edgecolors='black')

    if step_number is not None:
        plt.text(0.02, 0.98, f"Step: {step_number}",
                 transform=plt.gca().transAxes, fontsize=14, fontweight='bold')

    for i in range(n):
        plt.text(pos[i][0], pos[i][1], str(i),
                 fontsize=12, ha='center', va='center', fontweight='bold')

    buf = io.BytesIO()
    plt.tight_layout()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    plt.close()
    return img_str

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/simulate', methods=['POST'])
def simulate():
    data = request.json
    n = data['nodes']
    edges_raw = data['edges']
    algo = data['algorithm']
    max_colours = data.get('max_colours', n)

    pop_size = data.get('pop_size', 20)
    generations = data.get('generations', 50)
    mutation_rate = data.get('mutation_rate', 0.1)

    graph = [[0]*n for _ in range(n)]
    for u, v in edges_raw:
        graph[u][v] = graph[v][u] = 1

    start = time.perf_counter()

    if algo == 'backtracking':
        sol, _, m, steps = find_optimal_backtracking(graph, max_colours)
        final = sol if sol else [-1]*n
        solution_data = {
            "algorithm": "Backtracking",
            "solution": final,
            "time": time.perf_counter() - start,
            "steps": [(s, None) for s in steps],
            "min_colors": m if sol else 0
        }

    elif algo == 'backtracking_optimized':
        sol, m, steps, t = graph_colouring_optimized(graph, max_colours)
        final = sol if sol else [-1]*n
        solution_data = {
            "algorithm": "Optimized Backtracking",
            "solution": final,
            "time": t,
            "steps": steps,
            "min_colors": m if sol else 0
        }

    else:
        ca = CulturalAlgorithm(graph, pop_size, generations, mutation_rate, max_colours)
        sol, conflicts, t, steps = ca.evolve()
        
        final = sol 
        
        solution_data = {
            "algorithm": "Cultural Algorithm",
            "solution": final,
            "time": t,
            "steps": [(s, None) for s, _ in steps],
            "min_colors": calculate_chromatic_number(sol) if conflicts == 0 else 0
        }

    solution_data["conflicts"] = calculate_conflicts(graph, solution_data["solution"])
    solution_data["chromatic_number"] = calculate_chromatic_number(solution_data["solution"])

    return jsonify(solution_data)

@app.route('/step_image', methods=['POST'])
def step_image():
    data = request.json
    img = plot_graph_step(
        data['nodes'],
        data['edges'],
        data['coloring'],
        None,
        data.get('step_number', 0)
    )
    return jsonify({'image': img})

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Dec/2025 08:45:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Dec/2025 08:45:53] "GET /stars1.webp HTTP/1.1" 404 -
127.0.0.1 - - [15/Dec/2025 08:46:10] "POST /simulate HTTP/1.1" 200 -
C:\Users\ok\AppData\Local\Temp\ipykernel_22496\3121628681.py:64: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
127.0.0.1 - - [15/Dec/2025 08:46:10] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [15/Dec/2025 08:46:11] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [15/Dec/2025 08:46:12] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [15/Dec/2025 08:46:13] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [15/Dec/2025 08:46:14] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [15/Dec/2025 08:46:15] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [15/Dec/2025 08:46:15] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [15/Dec/2025 08:46:15] "POST /ste